In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np

import pandas as pd

from scipy.io import loadmat

from astropy.io import fits

import pandas as pd
import seaborn as sns

from lowfsc.automate import flt_chop_seq

from lowfsc import props, control

from lowfsc.data import DesignData
from lowfsc.spectral import StellarDatabase, LOWFS_BANDPASS, ThroughputDatabase, mk_bandpass
from lowfsc.automate import evaluate_estimator_noise, flt_chop_seq, plot_modes
from lowfsc.control import StateSpaceFilter

from lowfsc.emccd import EMCCD, sCMOS

from matplotlib import pyplot as plt

from tqdm import tqdm

In [ ]:
# reader note; I ran most all of this, then changed cam to cam2 in the main simulations and ran it again

root = Path('~/src/lowfssim-public/data')
root = root.expanduser()

mas2nm = 2.87

plt.style.use('bmh')

wvl = mk_bandpass(5)
mode='hlc' # or spec or wfov

# cam2 values taken from Gigajot QIS16 user manual
# dark current is 10x lower than manual; assume deeper cooling than 10C
# EMCCD noise = 0.0028 e-/px/s at -100C, but at 10C is ~100s of counts/sec
cam = EMCCD.cgi_camera()
cam2 = sCMOS(dark_current=0.0002, read_noise=0.53, bias=100, fwc=2000, conversion_gain=1/1.5, bits=14, exposure_time=441e-6)
wt = np.zeros(10)
mas2nm = 2.87

In [ ]:
# design test cases
test_cases_kwargs = [
    ('hlc_design',                      {}), # HLC band 1,
    ('hlc_band2_design',                {}),
    ('hlc_band3_design',                {}),
    ('hlc_band4_design',                {}),
    ('contributed_dual_pcwfs',          dict(sn=4, spot_depth=174)),
    ('contributed_pcwfsc_parametrized', {}),
    ('spec_design',                     {}),
    ('wfov_design',                     {})
]
titles = [
    'HLC Band 1',
    'HLC Band 2',
    'HLC Band 3',
    'HLC Band 4',
    'Ruane Dual PCWFS',
    'Classical PCWFS',
    'APLC-SPEC',
    'APLC-WFOV',
]

In [ ]:
# chop about absolute zero
# using 5 nm offsets
chop_zero = np.zeros(10)
chop_offsets = np.eye(10)*5
gains = np.diag(chop_offsets)
ref_star_config = (2.25, 'g0v', 1) # 1 -> 100 for EMCCD (LOWFS calibration knows about EM gain)
target_star_config = (5, 'g0v', 7) # 1 -> 750 for EMCCD

table = []
for factory_name, kwargs in test_cases_kwargs:
    # initialize model
    dd = getattr(DesignData, factory_name)(root, **kwargs)
    dd.seed_zernikes(range(2,12))  # Z2 to Z11
    
    # do Roman flight calibration sequence
    pak = flt_chop_seq(wvl, dd, ref_z=chop_zero, chop_zs=chop_offsets, gains=gains,
                       ref=ref_star_config, targ=target_star_config,
                       cam=cam2, style='') # default = APPROXTARGET; empty string is a true target star reference
    Rtarg = pak['R'] # R = reconstructor
    wref = pak['wref']
    wtarg = pak['wtarg'] # target star spectral weights
    
    cam.em_gain = target_star_config[2]
    
    # evaluate noise
    fig, sigmas = evaluate_estimator_noise(wvl, wtarg, cam2, dd, Rtarg)
    sigmas = sigmas[1:11] # just the Zernikes
    plt.close(fig)
    del fig
    
    # store results
    row = [factory_name, *sigmas]
    text_sigmas = [str(round(v, 2)) for v in sigmas]
    row_print = ','.join([factory_name, *text_sigmas])
    print(row_print)
    table.append(row)
#     break

In [ ]:
header = ['Rendition', *[f'Z{n}' for n in range(2,12)]]
df = pd.DataFrame(data=table, columns=header)
df2 = df.melt(id_vars=['Rendition'], value_vars=[f'Z{n}' for n in range(2,12)])

In [ ]:
plt.rcParams['font.size'] = 14 # 10

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

g = sns.barplot(data=df2, y='Rendition', x='value', hue='variable', ax=ax, palette='colorblind')
ax.get_legend().set_title('Term')
ax.set_yticklabels(titles)
ax.set(xlim=(0,6.25))
# g.set_xticklabels(rotation=90)
ax.set_xlabel(r'per 1kHz frame $\sigma$, nm')
ax.set_ylabel('Design');
plt.savefig('noises-different-masks-qis.png', dpi=200, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

g = sns.boxplot(data=df2, y='Rendition', x='value', palette='rocket')
# ax.get_legend().set_title('Term')
ax.set_yticklabels(titles)
ax.set(xlim=(0,6.25))
# g.set_xticklabels(rotation=90)
ax.set_xlabel(r'per 1kHz frame $\sigma$, nm')
ax.set_ylabel('Design');
plt.savefig('noises-different-masks-boxplot-qis.png', dpi=200, bbox_inches='tight')

In [ ]:
dd = DesignData.contributed_pcwfsc_parametrized(root)
dd.seed_zernikes(range(2,12))
pak = flt_chop_seq(wvl, dd, ref_z=chop_zero, chop_zs=chop_offsets, gains=gains,
                   ref=ref_star_config, targ=target_star_config,
                   cam=cam2, style='') # default = APPROXTARGET; empty string is a true target star reference

Rtarg = pak['R'] # R = reconstructor
wref = pak['wref']
wtarg = pak['wtarg'] # target star spectral weights
cam.em_gain = target_star_config[2]
ref = props.polychromatic(wvl, wtarg, dd)

noise_realizations = []
# this is on a 16" macbook pro laptop; ~2500 fps image + WFS simulation on CPU (!)
# diffraction model takes longer on CPU; ~60ms/crank (16 iter/s)
for _ in tqdm(range(1_000_000)):
    frame = cam2.expose(ref) # 1M frames = 1000 seconds
    estimates = Rtarg.estimate(frame)
    z5e = estimates[4]
    noise_realizations.append(z5e)

In [ ]:
Tmeas = 0.001 # 1ms/measurement (frame time, not integration time)
# now given a time series of random noise realizations, compute mean value vs integration
# pluck 200 logarithmically spaced sample lengths
calculate_at_samples = np.logspace(1, np.log10(len(noise_realizations)), 1000, dtype=int)
mean_values = [np.mean(noise_realizations[:i]) for i in calculate_at_samples]
mean_values = np.array(mean_values)

In [ ]:
plt.rcParams['font.size'] = 10 # 10

In [ ]:
np.set_printoptions(precision=3, suppress=True, linewidth=120)

# this is brute force numerical
# fig, sigmas = evaluate_estimator_noise(wvl, wtarg, cam2, dd, Rtarg)
# # sigmas = sigmas[1:11] # just the Zernikes
# plt.close(fig)
# del fig

Rtarg = pak['R'] # R = reconstructor
wref = pak['wref']
wtarg = pak['wtarg'] # target star spectral weights
cam.em_gain = target_star_config[2]
# ref = props.polychromatic(wvl, wtarg, dd)

P = Rtarg.P
# var_px = (cam2.expose(ref).astype(float) - cam2.expose(np.zeros_like(ref))) * 2
# var_px = (ref+cam2.dark_current)*cam2.exposure_time*cam2.conversion_gain + cam2.read_noise**2
cam2.read_noise = 0   # 0.53
cam2.dark_current = 0 # 0.0002

cam.dark_current = 0 # 0.0028
cam.read_noise = 0 # 200
cam.cic = 0 # 0.02

framestack = cam2.expose(ref, 10_000)
var_px = framestack.var(axis=0)
print(var_px[25,25])
# var_px2 = ((ref[25,25]+cam2.dark_current)*cam2.exposure_time + cam2.read_noise**2)/cam2.conversion_gain

# var_px2 = ref[25,25]*cam.exposure_time*(cam.em_gain/cam.conversion_gain)**2*2
var_px2 = ref[25,25]*cam2.exposure_time/cam2.conversion_gain/cam2.conversion_gain

print(var_px2)
# covar_mat = P @ np.diag(var_px.ravel()) @ P.T
# variances = np.diag(covar_mat)
# variances[1:11]**0.5, sigmas[1:11]

In [ ]:
csf = calculate_at_samples.astype(float)
expected_improvement = max(mean_values) / np.sqrt(csf)
expected_improvement = mean_values[0] / np.sqrt(csf)
plt.loglog(csf*Tmeas, abs(mean_values-mean_values[-1]), csf*Tmeas, abs(expected_improvement))
ax = plt.gca()
ax.set(ylim=(1e-5,1))
ax.axhline(0.5e-3, ls='-.', lw=1.5, c='k')
ax.text(1.1e-2, 1e-3, '0.5pm REQ')
ax.set(ylabel='Mean value, nm', xlabel='# of seconds of averaging')
plt.savefig('required-integration-time-loglog-qis.png', dpi=200, bbox_inches='tight')

In [ ]:
csf = calculate_at_samples.astype(float)
expected_improvement = max(mean_values) / np.sqrt(csf)
expected_improvement = mean_values[0] / np.sqrt(csf)
plt.loglog(csf*Tmeas, abs(mean_values), csf*Tmeas, abs(expected_improvement))
ax = plt.gca()
ax.set(ylim=(1e-5,1))
ax.axhline(0.5e-3, ls='-.', lw=1.5, c='k')
ax.text(1.1e-2, 1e-3, '0.5pm REQ')
ax.set(ylabel='Mean value, nm', xlabel='# of seconds of averaging')
plt.savefig('required-integration-time-loglog-no-bias-comp-qis.png', dpi=200, bbox_inches='tight')

In [ ]:
# this is an alternative to flt_chop_seq that does not have to synthesize the 1000s of frames,
# for the sake of my laptop.  Mostly copy-pasted; uses a fully analytic (non-numeric) evaluation
# of noise propagation
from lowfsc.automate import default_sd, default_td, chop_bipolar
from lowfsc.reconstruction import synthesize_pupil_shear, prepare_Zmm, ReconstructorV2pt5 as RV25
td, sd = default_td, default_sd
def simplified_chopping_seq_and_noise(wvl, dd, ref_z, chop_zs, gains, ref, targ, cam):
    ref_z_target = ref_z
    Vref, Cref, Gref = ref
    Vtarg, Ctarg, Gtarg = targ

    if hasattr(cam, 'em_gain'):  # support sCMOS
        oldgain = cam.em_gain
        cam.em_gain = Gref
    else:
        Gref, Gtarg = 1, 1  # avoid calcs below going awry based on bogus gain that doesn't exist

    # spectral weight vector computations
    throughput = td(mode, wvl)

    weights = sd(Cref, wvl)
    fudge = sd.sparsity_fudge_factor(Cref, wvl)
    v = 10 ** (-Vref/2.5)
    wref = weights * (fudge*v*throughput)

    weights = sd(Ctarg, wvl)
    fudge = sd.sparsity_fudge_factor(Ctarg, wvl)
    v = 10 ** (-Vtarg/2.5)
    wtarg = weights * (fudge*v*throughput)


    refref = props.polychromatic(wvl, wref, dd, ref_z)
    reftarg = props.polychromatic(wvl, wtarg, dd, ref_z_target)
    _dark = np.zeros_like(refref) + cam.bias/cam.conversion_gain
    
    refref *= cam.exposure_time/cam.conversion_gain
    reftarg *= cam.exposure_time/cam.conversion_gain
    
    Sref = float(refref.sum())
    Starg = float(reftarg.sum())
    Sratio = Starg/Sref
    
    # reference star chopping
    # this is different to flt_chop_seq in that the chops are noiseless and unquantized
    diffs, *_ = chop_bipolar(wvl, wref, dd, ref_z, chop_zs, cam=None, nframes_avg=None)
    for (d, g) in zip(diffs, gains):  # gain here is not EM gain, but sensing gain
        d *= (cam.exposure_time*Sratio)/(cam.conversion_gain*g)
    
    fmtarg = reftarg
    darktarg = _dark
    
    CHOP_SHEAR_PX = 0.038
    sy = synthesize_pupil_shear(fmtarg, CHOP_SHEAR_PX, 1)
    sx = synthesize_pupil_shear(fmtarg, CHOP_SHEAR_PX, 0)
    
    fmtark = reftarg
    darktarg = _dark
    
    mask = np.ones_like(sy)
    mask[0,  :] = 0  # NOQA
    mask[-1, :] = 0
    mask[:,  0] = 0
    mask[:, -1] = 0
    zmm = prepare_Zmm(diffs, fmtarg, (sx, sy), mask)
    R = RV25(zmm, reftarg, darktarg)
    
    # now noise
    P = R.P
    
    # reftarg already has exposure time and one factor of K-gain baked in
    expectation_variance = (reftarg*cam.conversion_gain + cam.dark_current*cam.exposure_time + cam.read_noise**2)/cam.conversion_gain/cam.conversion_gain
    
    covar = P @ np.diag(expectation_variance.ravel()) @ P.T
    var = np.diag(covar)
    sigma = np.sqrt(var)
    return sigma

In [ ]:
chop_zero = np.zeros(10)
chop_offsets = np.eye(10)*5
gains = np.diag(chop_offsets)
ref_star_config = (2.25, 'g0v', 100)
target_star_config = (5, 'g0v', 750)


dd = DesignData.contributed_pcwfsc_parametrized(root)
dd.seed_zernikes(range(2,12))

bin_factors = [8,4,2,1]
resolutions = [400/b for b in bin_factors]

table = []
for binf, res in zip(bin_factors, resolutions):
    dd.bin_factor = binf
    sigmas = simplified_chopping_seq_and_noise(wvl, dd, chop_zero, chop_offsets, gains, ref_star_config, target_star_config, cam2)
    sigmas = sigmas[1:11]
    
    # store results
    row = [res, *sigmas]
    text_sigmas = [str(round(v, 2)) for v in sigmas]
    row_print = ','.join([str(res), *text_sigmas])
    print(row_print)
    table.append(row)
#     break

In [ ]:
xx, yy = df['Detector Resolution'], df['Z11']
coefs = np.polyfit(xx, yy, deg=2)
xxx = np.linspace(50,400, 200)
zzz = np.polyval(coefs, xxx)
plt.plot(xxx, zzz, c='C1')
plt.scatter(xx, yy, s=55)
ax = plt.gca()
ax.set(xlabel='Detector Resolution, px', ylabel=r'per 1kHz frame $\sigma$, nm', ylim=(0,1.5))
plt.savefig('dependence-of-noise-on-image-size.png', dpi=200, bbox_inches='tight')